In [1]:
import json
import warnings
from functools import partial
from pathlib import Path

import geopandas as gpd
import numpy as np
import pandas as pd
from tqdm import tqdm

import nird.road_recovery as func
import nird.road_functions as post_func
from nird.utils import load_config, get_flow_on_edges

from tabulate import tabulate

warnings.simplefilter("ignore")
base_path = Path(load_config()["paths"]["base_path"])
tqdm.pandas()

## Traffic flow disruption and rerouting analysis

In this example, we estimated the indirect costs of traffic delays and disruptions by simulating changes in traffic flow rerouting. This included tracking the evolving number of isolated trips and rerouting costs—such as time-equivalent delays, fuel consumption, and tolls—as the functionality of both bridge and non-bridge road assets was progressively restored on a daily basis.

In [2]:
# Load recovery rate data
bridge_recovery_dict, road_recovery_dict = post_func.load_recovery_dicts(base_path)

In [3]:
# Load network parameters
with open(base_path / "parameters" / "flow_cap_plph_dict.json", "r") as f:
    flow_capacity_dict = json.load(f)
with open(base_path / "parameters" / "flow_breakpoint_dict.json", "r") as f:
    flow_breakpoint_dict = json.load(f)
partial_speed_flow_func = partial(
    func.speed_flow_func, flow_breakpoint_dict=flow_breakpoint_dict
)

In [4]:
# Load road links with damages
road_links_with_damages = gpd.read_parquet(
    base_path.parent / "outputs"
        / "disruption_analysis"
        / "20241229"
        / "30"
        / "links"
        / "road_links_17.gpq"  # Thames Lloyd's RDS
)
road_links_with_damages["designed_capacity"] = road_links_with_damages.combined_label.map(flow_capacity_dict) * road_links_with_damages.lanes * 24

In [5]:
road_links_with_damages.head()

,from_id,to_id,e_id,fictitious,road_classification,road_function,form_of_way,road_classification_number,name_1,name_1_lang,...,damage_level_max,free_flow_speeds,initial_flow_speeds,min_flow_speeds,max_speed,current_capacity,current_speed,current_flow,combined_label,designed_capacity
0,67C5F67E-3D2E-4EAE-8546-99A0B55D8094,32A6AE75-AAD5-456E-8CF4-B7012B520D4E,roade_0,False,A Road,A Road,Single Carriageway,A4017,Bromley Heath Road,None,...,no,37.0,37.0,0.2,37.0,138708,37.0,5292,A_single,144000
1,33FEA706-7956-41E3-9825-11D6831837DF,648DD533-65BD-4148-93ED-683D6D0E1C40,roade_1,False,A Road,A Road,Single Carriageway,A85,None,None,...,no,37.0,37.0,0.2,37.0,71904,37.0,96,A_single,72000
2,10DE2E65-4BED-4F49-9ED5-E4D9D872079B,C58D254E-B492-4C2A-8863-ECD569F30988,roade_2,False,A Road,A Road,Roundabout,A642,None,None,...,no,45.0,45.0,0.2,45.0,72000,45.0,0,A_dual,72000
3,E64AE8DD-A4CE-4D9D-97FB-757A0B8E8571,283C5F76-5CFC-4EAA-90EA-F279A86DF847,roade_3,False,A Road,A Road,Single Carriageway,A705,Redhouse Road,None,...,no,37.0,37.0,0.2,37.0,70498,37.0,1502,A_single,72000
4,62A1E3AE-606C-4E9E-9375-9841B262ADB2,E8A899D2-7509-452E-BAA6-2B1A1372802A,roade_4,False,A Road,A Road,Single Carriageway,A4075,None,None,...,no,37.0,37.0,0.2,37.0,71820,37.0,180,A_single,72000


In [6]:
# Load OD path file
od_path_file = pd.read_parquet(
    base_path.parent / "outputs" / "base_scenario" / "odpfc_32p.pq", # with only major flows (od flow > 2 for return trips)
    engine="fastparquet",
)

In [7]:
od_path_file.shape

(1767524, 7)

In [8]:
# Randomly sample 10% of the data for testing
od_path_file = od_path_file.sample(n=1000, random_state=42, ignore_index=True)

In [9]:
# Identify the disrupted links
disrupted_links = (
    road_links_with_damages.loc[
        (road_links_with_damages.max_speed < road_links_with_damages.current_speed)
        | (road_links_with_damages.damage_level_max == "extensive")
        | (road_links_with_damages.damage_level_max == "severe"),
        "e_id",
    ]
    .unique()
    .tolist()
)
partial_have_common_items = partial(post_func.have_common_items, list2=disrupted_links)

In [10]:
# Run road recovery and traffic flow rerouting analysis loop (for 110 days)
cDict = {}
for day in range(111):
    if day not in [0, 1, 36, 51]: # only simulate days 0, 1, 36, and 51 for demonstration

        continue
    print(f"Rerouting Analysis on D-{day}...")
    # update edge capacities
    road_links_with_damages["current_capacity"] = road_links_with_damages.progress_apply(
        lambda row: (
            post_func.bridge_recovery(
                day,
                row["damage_level_max"],
                row["designed_capacity"],
                row["current_capacity"],
                bridge_recovery_dict,
            )
            if row["road_label"] == "bridge"
            else (
                post_func.ordinary_road_recovery(
                    day,
                    row["damage_level_max"],
                    row["designed_capacity"],
                    row["current_capacity"],
                    road_recovery_dict,
                )
            )
        ),
        axis=1,
    )

    # Update the disrupted OD paths
    od_path_file["disrupted_links"] = np.vectorize(partial_have_common_items)(
        od_path_file.path
    )
    od_path_file["disrupted_links"] = od_path_file["disrupted_links"].apply(list)
    current_capacity_dict = road_links_with_damages.set_index("e_id")[
        "current_capacity"
    ].to_dict()
    od_path_file["capacities_of_disrupted_links"] = od_path_file[
        "disrupted_links"
    ].apply(lambda x: [current_capacity_dict.get(xi, 0) for xi in x])

    od_path_file["min_capacities_of_disrupted_links"] = od_path_file[
        "capacities_of_disrupted_links"
    ].apply(lambda x: min(x) if len(x) > 0 else np.nan)

    disrupted_od = od_path_file.loc[
        od_path_file["min_capacities_of_disrupted_links"].notnull()
    ]
    disrupted_od["flow"] = np.maximum(
        0,
        disrupted_od["flow"] - disrupted_od["min_capacities_of_disrupted_links"],
    )

    # Adjust road flows
    road_links_with_damages["disrupted_flow"] = 0.0
    disrupted_edge_flow = get_flow_on_edges(disrupted_od, "e_id", "path", "flow")
    disrupted_edge_flow.rename(columns={"flow": "disrupted_flow"}, inplace=True)
    road_links_with_damages = road_links_with_damages.set_index("e_id")
    road_links_with_damages.update(disrupted_edge_flow.set_index("e_id")["disrupted_flow"])
    road_links_with_damages = road_links_with_damages.reset_index()
    road_links_with_damages["acc_capacity"] = road_links_with_damages["current_capacity"] + np.where(
        road_links_with_damages["damage_level_max"].isin(["extensive", "severe"]),
        0,
        road_links_with_damages["disrupted_flow"],
    )

    # Flow speed recalculation
    road_links_with_damages["acc_speed"] = np.vectorize(partial_speed_flow_func)(
        road_links_with_damages["combined_label"],
        road_links_with_damages["current_flow"],
        road_links_with_damages["initial_flow_speeds"],
        road_links_with_damages["min_flow_speeds"],
    )
    if day == 0:
        road_links_with_damages["acc_speed"] = road_links_with_damages[["current_speed", "max_speed"]].min(
            axis=1
        )

    # initial key variables
    road_links_with_damages["acc_flow"] = 0
    disrupted_od.rename(columns={"flow": "Car21"}, inplace=True)
    # disrupted_od = disrupted_od.head(100)  # for debug

    # create network
    valid_road_links = road_links_with_damages[
        (road_links_with_damages["acc_capacity"] > 0) & (road_links_with_damages["acc_speed"] > 0)
    ].reset_index(drop=True)
    network = func.create_igraph_network(valid_road_links)

    # run flow rerouting analysis
    _, isolation, _, cList = func.network_flow_model(
        valid_road_links,
        network,
        disrupted_od,
        flow_breakpoint_dict,
        num_of_cpu=1, # the number of CPU cores to use for parallel processing
    )
    cDict[day] = cList
    road_links_with_damages = road_links_with_damages.set_index("e_id")
    road_links_with_damages.update(valid_road_links.set_index("e_id")["acc_flow"])
    road_links_with_damages = road_links_with_damages.reset_index()

    # identify isolated flows (with no accessible routes)
    isolation_df = pd.DataFrame(
        isolation,
        columns=[
            "origin_node",
            "destination_node",
            "Car21",
        ],
    )
    print("Number of isolated flows (passengers): ")
    print(tabulate(isolation_df.head(), headers='keys', tablefmt='psql'))

# record the cost data for 110 days
cost_df = pd.DataFrame.from_dict(
    cDict, orient="index", columns=["time", "fuel", "toll", "total"]
).reset_index()
cost_df.rename(columns={"index": "day"}, inplace=True)
print("Total costs (£): ")
print(tabulate(cost_df.head(), headers='keys', tablefmt='psql'))


Rerouting Analysis on D-0...


100%|██████████| 505750/505750 [00:04<00:00, 119131.85it/s]


The initial supply is 12.0
The initial number of edges in the network: 504370
The initial number of origins: 11
The initial number of destinations: 11
No.1 iteration starts:
The least-cost path flow allocation time: 3.0449864864349365.
The remaining number of origins: 8
The remaining number of destinations: 8
The maximum amount of edge overflow: -64530.0
Iteration stops: there is no edge overflow! with 66.66666666666666% flows sent to the network!
The flow simulation is completed!
total travel cost is (£): 108.31833127497161
total time-equiv cost is (£): 79.28225932041721
total operating cost is (£): 29.0360719545544
total toll cost is (£): 0.0
Number of isolated flows (passengers): 
+----+--------------------------------------+--------------------------------------+---------+
|    | origin_node                          | destination_node                     |   Car21 |
|----+--------------------------------------+--------------------------------------+---------|
|  0 | 59390193-B768-4

100%|██████████| 505750/505750 [00:03<00:00, 129331.59it/s]


The initial supply is 8.0
The initial number of edges in the network: 505710
The initial number of origins: 11
The initial number of destinations: 11
No.1 iteration starts:
The least-cost path flow allocation time: 3.2228353023529053.
The remaining number of origins: 11
The remaining number of destinations: 11
The maximum amount of edge overflow: -64722.0
Iteration stops: there is no edge overflow! with 100.0% flows sent to the network!
The flow simulation is completed!
total travel cost is (£): 75.88857228866547
total time-equiv cost is (£): 54.610913573528194
total operating cost is (£): 21.27765871513727
total toll cost is (£): 0.0
Number of isolated flows (passengers): 
+---------------+--------------------+---------+
| origin_node   | destination_node   | Car21   |
|---------------+--------------------+---------|
+---------------+--------------------+---------+
Rerouting Analysis on D-36...


100%|██████████| 505750/505750 [00:04<00:00, 125740.52it/s]


The initial supply is 0.0
The initial number of edges in the network: 505748
The initial number of origins: 11
The initial number of destinations: 11
The flow simulation is completed!
total travel cost is (£): 0
total time-equiv cost is (£): 0
total operating cost is (£): 0
total toll cost is (£): 0
Number of isolated flows (passengers): 
+---------------+--------------------+---------+
| origin_node   | destination_node   | Car21   |
|---------------+--------------------+---------|
+---------------+--------------------+---------+
Rerouting Analysis on D-51...


100%|██████████| 505750/505750 [00:03<00:00, 128587.87it/s]


The initial supply is 0.0
The initial number of edges in the network: 505750
The initial number of origins: 11
The initial number of destinations: 11
The flow simulation is completed!
total travel cost is (£): 0
total time-equiv cost is (£): 0
total operating cost is (£): 0
total toll cost is (£): 0
Number of isolated flows (passengers): 
+---------------+--------------------+---------+
| origin_node   | destination_node   | Car21   |
|---------------+--------------------+---------|
+---------------+--------------------+---------+
Total costs (£): 
+----+-------+---------+---------+--------+----------+
|    |   day |    time |    fuel |   toll |    total |
|----+-------+---------+---------+--------+----------|
|  0 |     0 | 79.2823 | 29.0361 |      0 | 108.318  |
|  1 |     1 | 54.6109 | 21.2777 |      0 |  75.8886 |
|  2 |    36 |  0      |  0      |      0 |   0      |
|  3 |    51 |  0      |  0      |      0 |   0      |
+----+-------+---------+---------+--------+----------+
